### bulk ExpressionSet

In [1]:
library(SummarizedExperiment)

Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min


Loading required package: S4Vecto

In [2]:
Matrix <- read.table("/lustre/scratch117/cellgen/team205/rl20/CTCL/deconvolution/GSE121212_readcount_rmdup.txt", 
                     check.names=F) ### count matrix

In [3]:
head(Matrix);dim(Matrix)

,AD_004_lesional,AD_004_non-lesional,AD_005_lesional,AD_005_non-lesional,AD_006_lesional,AD_006_non-lesional,AD_007_lesional,AD_007_non-lesional,AD_009_lesional,AD_009_non-lesional,⋯,AD_033_chronic_lesion,AD_034_non-lesional,AD_034_chronic_lesion,AD_032_chronic_lesion,AD_035_non-lesional,AD_035_chronic_lesion,AD_036_non-lesional,AD_036_chronic_lesion,AD_037_non-lesional,AD_037_chronic_lesion
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
5S_rRNA,0,3,9,12,2,3,6,4,2,5,⋯,1,2,3,3,3,4,3,5,6,2
7SK,9,6,25,25,7,8,9,8,6,15,⋯,3,8,2,5,12,5,7,5,8,3
A1BG,2,0,4,3,1,1,0,0,4,0,⋯,1,1,2,0,1,2,0,0,1,1
A1BG-AS1,5,0,11,8,7,8,13,2,6,19,⋯,4,0,7,7,4,4,10,13,8,4
A1CF,2,3,2,8,1,2,0,5,1,5,⋯,3,0,1,0,0,2,0,0,1,0
A2M,308,353,2544,1366,978,406,2761,928,2796,1891,⋯,1463,1391,1714,1998,683,1154,2361,2374,1040,2755


[1] 31362   147

In [4]:
is_genename_uniq <- !duplicated(rownames(Matrix))
Matrix <- Matrix[is_genename_uniq, ]
dim(Matrix)

[1] 31362   147

In [5]:
tt <- apply(Matrix, 2, sum)
del <- which(tt==0)
del

named integer(0)

In [6]:
fdat_df <- data.frame(gene_name=rownames(Matrix), stringsAsFactors=F)
rownames(fdat_df) <- rownames(Matrix)
metadata_fdat_df <- data.frame(labelDescription= c("gene name"), row.names=c("gene_name"))
head(fdat_df)

,gene_name
,<chr>
5S_rRNA,5S_rRNA
7SK,7SK
A1BG,A1BG
A1BG-AS1,A1BG-AS1
A1CF,A1CF
A2M,A2M


In [7]:
metadata <- read.table("/lustre/scratch117/cellgen/team205/rl20/CTCL/deconvolution/GSE121212_meta.xls", 
                       check.names=F, sep="\t", header = T)
head(metadata)

,sample_type,condition
,<chr>,<chr>
AD_004_lesional,AD_L,AD
AD_004_non-lesional,AD_NL,AD
AD_005_lesional,AD_L,AD
AD_005_non-lesional,AD_NL,AD
AD_006_lesional,AD_L,AD
AD_006_non-lesional,AD_NL,AD


In [8]:
pdat_df <- data.frame(sample_type = metadata$sample_type, 
                      condition = metadata$condition,
                      stringsAsFactors = F)
rownames(pdat_df) <- rownames(metadata)
head(pdat_df)

,sample_type,condition
,<chr>,<chr>
AD_004_lesional,AD_L,AD
AD_004_non-lesional,AD_NL,AD
AD_005_lesional,AD_L,AD
AD_005_non-lesional,AD_NL,AD
AD_006_lesional,AD_L,AD
AD_006_non-lesional,AD_NL,AD


In [9]:
metadata_pdat_df <- data.frame(labelDescription= c("lesion_nonlesion", "Condition"), 
                               row.names=c("sample_type", "condition"))

In [10]:
metadata_pdat_df

,labelDescription
,<chr>
sample_type,lesion_nonlesion
condition,Condition


In [11]:
Matrix <- Matrix[ ,rownames(pdat_df)] ### the order of rownames(pdat_df) and colnames(Matrix) might be different

In [12]:
all.equal(colnames(Matrix), rownames(pdat_df))

[1] TRUE

In [13]:
bulk_eset <- ExpressionSet(
  assayData = data.matrix(Matrix),
  phenoData=new("AnnotatedDataFrame", data = pdat_df, varMetadata = metadata_pdat_df),
  featureData=new("AnnotatedDataFrame", data = fdat_df, varMetadata = metadata_fdat_df))

In [14]:
bulk_eset

ExpressionSet (storageMode: lockedEnvironment)
assayData: 31362 features, 147 samples 
  element names: exprs 
protocolData: none
phenoData
  sampleNames: AD_004_lesional AD_004_non-lesional ...
    AD_037_chronic_lesion (147 total)
  varLabels: sample_type condition
  varMetadata: labelDescription
featureData
  featureNames: 5S_rRNA 7SK ... snoZ5 (31362 total)
  fvarLabels: gene_name
  fvarMetadata: labelDescription
experimentData: use 'experimentData(object)'
Annotation:  

In [17]:
saveRDS(bulk_eset, file="/lustre/scratch117/cellgen/team205/rl20/CTCL/deconvolution/GSE121212_readcount_ExpressionSet.rds")

### Single cell ExpressionSet

In [2]:
library(sceasy)
library(reticulate)
library(anndata)
library(Seurat)
library(BisqueRNA)
library(Biobase)
library(Matrix)

Loading required package: reticulate

Attaching SeuratObject


Attaching package: ‘Seurat’


The following object is masked from ‘package:SummarizedExperiment’:

    Assays



Attaching package: ‘Matrix’


The following object is masked from ‘package:S4Vectors’:

    expand




In [3]:
h5ad_file <- "/lustre/scratch126/cellgen/team205/rl20/CTCL/object_revision/All_samples_final_20240707_sub0.08_for_deconv.h5ad"
sdata <- read_h5ad(h5ad_file)
seurat_object <- CreateSeuratObject(counts = t(as.matrix(sdata$X)), meta.data = sdata$obs)

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 7.3 GiB”


In [4]:
seurat_object

An object of class Seurat 
15777 features across 62199 samples within 1 assay 
Active assay: RNA (15777 features, 0 variable features)

In [5]:
gene_exprs.matrix <- seurat_object@assays$RNA@counts
gene_exprs.matrix[1:20,1:20]


  [[ suppressing 20 column names ‘AAACCTGCATCACAAC-0_CTCL1_CTCL1_CTCL1’, ‘AAACCTGCATGTAAGA-0_CTCL1_CTCL1_CTCL1’, ‘AAACGGGGTCGACTGC-0_CTCL1_CTCL1_CTCL1’ ... ]]



20 x 20 sparse Matrix of class "dgCMatrix"
                                                
SAMD11   . . . . . . . . . . . . . . . . . . . .
NOC2L    1 1 . . 2 . . 1 . . . 3 1 . . . . . . .
KLHL17   . . . . . . . . . . . . . . . . . . . .
PLEKHN1  2 . . . . . . . 1 . . 1 . . . . . . . .
HES4     . . 1 . . . . . . . 2 . . . . . 1 . . .
ISG15    . . . . . . . 1 . . . . . . . . . . . .
AGRN     . . . . . . . . . 1 . . . . 1 . . . . .
RNF223   . . . . . . . . . . . . . . . . . . . .
C1orf159 . . . . . . . . . . 1 . . . . . . . . .
TTLL10   . . . . . . . . . . . . . . . . . . . .
TNFRSF18 3 1 1 . 1 . . 5 . . . 1 3 . . 1 . . . .
TNFRSF4  . . . . . . . 3 . 1 . . . . . . . . . .
SDF4     6 1 5 1 1 3 . 2 . . . 3 . 2 2 1 1 . . .
B3GALT6  . . . . . . . . . . . . . . . . . . . .
C1QTNF12 . . 1 . . . . . . . . . . . 1 . . . 1 .
UBE2J2   . 1 . . 3 . . 1 . . 4 1 . 1 . . 2 . . .
SCNN1D   . . 1 . . . . . . . . . . . . . . . . .
ACAP3    . . . . . 1 . . . . . . . . . . . . . .
PUSL1    . . . . . . . . 1

In [6]:
head(seurat_object@meta.data)

,orig.ident,nCount_RNA,nFeature_RNA,batch,n_counts,n_genes,percent_mito,donor,tech,sample_type,⋯,broad_ct,ctpredicted_labels,ctover_clustering,ctmajority_voting,ctconf_score,cell_type,site,tissue,Sex,type
,<fct>,<dbl>,<int>,<fct>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>
AAACCTGCATCACAAC-0_CTCL1_CTCL1_CTCL1,SeuratProject,14602,3142,0,27577,3474,0.0011241252,CTCL1,10x,CTCL,⋯,Keratinocytes,Differentiated_KC*,286,Differentiated_KC,0.96744886,Differentiated_KC,lesion,Epidermis,Female,NA
AAACCTGCATGTAAGA-0_CTCL1_CTCL1_CTCL1,SeuratProject,13463,3311,0,21021,3642,0.0004281433,CTCL1,10x,CTCL,⋯,Keratinocytes,Differentiated_KC*,492,Differentiated_KC*,0.13851590,Differentiated_KC*,lesion,Epidermis,Female,NA
AAACGGGGTCGACTGC-0_CTCL1_CTCL1_CTCL1,SeuratProject,14551,3472,0,27837,3824,0.0024787153,CTCL1,10x,CTCL,⋯,Keratinocytes,Differentiated_KC*,535,Differentiated_KC,0.01321663,Differentiated_KC,lesion,Epidermis,Female,NA
AAAGCAAAGCGTAATA-0_CTCL1_CTCL1_CTCL1,SeuratProject,11215,2632,0,17545,2943,0.0056426334,CTCL1,10x,CTCL,⋯,Keratinocytes,Differentiated_KC*,298,Differentiated_KC*,0.90835925,Differentiated_KC*,lesion,Epidermis,Female,NA
AAAGCAAGTCCTGCTT-0_CTCL1_CTCL1_CTCL1,SeuratProject,6150,2384,0,9136,2665,0.0010945710,CTCL1,10x,CTCL,⋯,Keratinocytes,VE2,368,Differentiated_KC*,0.04688790,Differentiated_KC*,lesion,Epidermis,Female,NA
AAAGCAATCCCATTTA-0_CTCL1_CTCL1_CTCL1,SeuratProject,5296,1860,0,11803,2101,0.0155045325,CTCL1,10x,CTCL,⋯,Keratinocytes,Differentiated_KC,438,Differentiated_KC,0.67155350,Differentiated_KC,lesion,Epidermis,Female,NA


In [7]:
#pheno.matrix <- seurat_object@meta.data[,c("donor_id","broad_cell_type","anno")]
pheno.matrix <- seurat_object@meta.data[,c("donor","cell_type")]
head(pheno.matrix)

,donor,cell_type
,<fct>,<fct>
AAACCTGCATCACAAC-0_CTCL1_CTCL1_CTCL1,CTCL1,Differentiated_KC
AAACCTGCATGTAAGA-0_CTCL1_CTCL1_CTCL1,CTCL1,Differentiated_KC*
AAACGGGGTCGACTGC-0_CTCL1_CTCL1_CTCL1,CTCL1,Differentiated_KC
AAAGCAAAGCGTAATA-0_CTCL1_CTCL1_CTCL1,CTCL1,Differentiated_KC*
AAAGCAAGTCCTGCTT-0_CTCL1_CTCL1_CTCL1,CTCL1,Differentiated_KC*
AAAGCAATCCCATTTA-0_CTCL1_CTCL1_CTCL1,CTCL1,Differentiated_KC


In [8]:
metadata <- data.frame(labelDescription= c("Donor ID", "Cell type"), 
                   row.names=c("donor", "cell_type"))
metadata

,labelDescription
,<chr>
donor,Donor ID
cell_type,Cell type


In [9]:
library(Matrix) 
#expr_matrix <- integrated@assays$RNA@counts #sparsematrix
gene_exprs.matrix <- gene_exprs.matrix[tabulate(summary(gene_exprs.matrix)$i) != 0, , drop = FALSE] #remove any feature without a single count
#expr_matrix <- as.matrix(gene_exprs.matrix) #convert to matrix

In [10]:
dim(gene_exprs.matrix)

[1] 15767 62199

In [28]:
expr_matrix_den <- SCOPfunctions::utils_big_as.matrix(gene_exprs.matrix, n_slices_init = 1, verbose = T) ## for large matrix

n_slices=1

converting slice 1/1

columns 1:55815

cbind dense submatrices



In [11]:
SC.eset = ExpressionSet(assayData = data.matrix(gene_exprs.matrix), 
                        phenoData =  new("AnnotatedDataFrame", data = pheno.matrix, varMetadata = metadata))
#SC.eset = ExpressionSet(assayData = expr_matrix_den, 
#                        phenoData =  new("AnnotatedDataFrame", data = pheno.matrix, varMetadata = metadata)) ## for large matrix

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 7.3 GiB”


In [12]:
SC.eset

ExpressionSet (storageMode: lockedEnvironment)
assayData: 15767 features, 62199 samples 
  element names: exprs 
protocolData: none
phenoData
  sampleNames: AAACCTGCATCACAAC-0_CTCL1_CTCL1_CTCL1
    AAACCTGCATGTAAGA-0_CTCL1_CTCL1_CTCL1 ... GACGTGCTCACATACG-92_S5
    (62199 total)
  varLabels: donor cell_type
  varMetadata: labelDescription
featureData: none
experimentData: use 'experimentData(object)'
Annotation:  

In [ ]:
saveRDS(SC.eset, file="/lustre/scratch126/cellgen/team205/rl20/CTCL/object_revision/All_samples_final_20240707_sub0.08_for_deconv_ExpressionSet.rds")